In [1]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path

sys.path.append(os.path.abspath(".."))

from utils.model_ann.ann import build_index, search

# Load Dataset

In [2]:
DOMAIN = "fashion"

current_dir = os.path.abspath(os.curdir)
base_dir = "/".join(current_dir.split("/")[:-1])

In [3]:
dataset_dir = Path(base_dir).joinpath(f"data/dataset/{DOMAIN}/interactions")
df = pd.read_parquet(dataset_dir)
df.head()

,user_id,item_id,timestamp,action,age,gender,title,color,style,fit,material,season,sleeve,category
0,1,3507,2025-09-21 03:01:00,click,46,F,슬림핏 실루엣의 데님 오버롤,레드,스포티,슬림핏,데님,겨울,롱슬리브,오버롤
1,1,9867,2025-09-21 03:02:00,click,46,F,봄 슬림핏 레드 나일론 롱슬리브 오버롤,레드,스포티,슬림핏,나일론,봄,롱슬리브,오버롤
2,1,3471,2025-09-02 00:05:00,click,46,F,"간절기 시즌, 레트로 무드의 오렌지 퍼 롱슬리브 루즈핏 원피스",오렌지,레트로,루즈핏,퍼,간절기,롱슬리브,원피스
3,1,6462,2025-09-02 00:09:00,click,46,F,"트렌디한 레트로룩, 슬림핏 원피스 in 여름",오렌지,레트로,슬림핏,스웨이드,여름,숏슬리브,원피스
4,1,441,2025-09-19 08:51:00,click,46,F,"봄 시즌, 캐주얼 무드의 와인 스웨이드 롱슬리브 루즈핏 슬랙스",와인,캐주얼,루즈핏,스웨이드,봄,롱슬리브,슬랙스


# Load Embedding Vectors

In [4]:
save_path = Path(base_dir).joinpath(f"data/model/{DOMAIN}/lightgcn")

In [5]:
df_user_vectors = pd.read_parquet(save_path.joinpath("user_vector.parquet"))
df_item_vectors = pd.read_parquet(save_path.joinpath("item_vector.parquet"))

In [6]:
user_id_maps = dict(zip(df_user_vectors["idx"], df_user_vectors["user_id"]))
item_id_maps = dict(zip(df_item_vectors["idx"], df_item_vectors["item_id"]))

In [7]:
user_vectors = np.array(df_user_vectors["vector_normalized"].tolist())
item_vectors = np.array(df_item_vectors["vector_normalized"].tolist())

user_vectors = np.array(user_vectors).astype(np.float32)
item_vectors = np.array(item_vectors).astype(np.float32)

# Vector Similarity Search

## User Similarity

In [8]:
# user_vectors: (num_users, dim) numpy float32 배열이라고 가정
user_index = build_index(user_vectors)
user_idx = 0
query_vector = user_vectors[user_idx]
result = search(query_vector, user_index, top_k=5)
print(result)  # 예: {1: 0.9, 2: 0.8, ...}

{0: 1.0000001192092896, 7088: 0.6814432740211487, 383: 0.5981080532073975, 9717: 0.5458018183708191, 4905: 0.500554084777832}


In [9]:
user_id = user_id_maps[user_idx]

print(f"User ID: {user_id}")
print("-----------------------------------")
for t in sorted(df[df["user_id"] == user_id]["title"].unique()):
    print(t)

User ID: 1
-----------------------------------
간절기 감성 크롭핏 퍼 청바지
간절기 시즌, 레트로 무드의 오렌지 퍼 롱슬리브 루즈핏 원피스
겨울 감성 루즈핏 가죽 청바지
겨울 감성 루즈핏 레이온 청바지
루즈핏 실루엣의 가죽 청바지
봄 슬림핏 레드 나일론 롱슬리브 오버롤
봄 시즌, 캐주얼 무드의 와인 스웨이드 롱슬리브 루즈핏 슬랙스
봄 한정 스웨이드 소재 와인 청바지
슬림핏 실루엣의 데님 오버롤
여름 한정 가죽 소재 화이트 오버롤
여름에 어울리는 화이트 오버롤
트렌디한 레트로룩, 슬림핏 원피스 in 여름
트렌디한 스트리트룩, 루즈핏 오버롤 in 봄
필수템! 스트리트 무드의 간절기용 오버롤
필수템! 스트리트 무드의 겨울용 오버롤
필수템! 캐주얼 무드의 겨울용 슬랙스
필수템! 캐주얼 무드의 봄용 슬랙스


In [10]:
similar_indies = list(result.keys())[1:]
similar_user_ids = [user_id_maps[i] for i in similar_indies]

for uid in similar_user_ids:
    print(f"\nUser ID: {uid}")
    print("-----------------------------------")
    for t in sorted(df[df["user_id"] == uid]["title"]):
        print(t)


User ID: 7089
-----------------------------------
가을 시즌, 미니멀 무드의 블랙 아크릴 7부 크롭핏 조끼
간절기 감성 크롭핏 아크릴 레깅스
간절기 감성 크롭핏 퍼 청바지
간절기 슬림핏 민트 울 숏 레깅스
간절기에 어울리는 레드 레깅스
겨울 감성 루즈핏 가죽 청바지
겨울 감성 루즈핏 레이온 청바지
겨울 크롭핏 레드 아크릴 숏슬리브 레깅스
루즈핏 실루엣의 가죽 청바지
모던 무드의 오렌지 레깅스
민트 컬러의 댄디 스타일 플리스 레깅스
봄 슬림핏 레드 레이온 민소매 레깅스
봄 한정 울 소재 민트 레깅스
봄 한정 플리스 소재 레드 레깅스
블랙 컬러의 미니멀 스타일 가죽 조끼
블랙 컬러의 미니멀 스타일 데님 조끼
여름 시즌, 모던 무드의 오렌지 레이온 롱 세미오버핏 레깅스
여름 크롭핏 민트 메쉬 7부 레깅스
필수템! 댄디 무드의 가을용 레깅스

User ID: 384
-----------------------------------
가을 세미오버핏 브라운 레이온 롱슬리브 카디건
간절기 한정 나일론 소재 올리브 오버롤
겨울 감성 크롭핏 메쉬 셔츠
겨울 한정 플리스 소재 브라운 카디건
러블리 무드의 네이비 와이드팬츠
슬림핏 실루엣의 아크릴 오버롤
여름 감성 슬림핏 플리스 오버롤
여름 시즌, 러블리 무드의 네이비 레이온 민소매 세미오버핏 와이드팬츠
여름 시즌, 모던 무드의 브라운 코튼 롱슬리브 레귤러핏 카디건
여름 시즌, 스트리트 무드의 화이트 레이온 민소매 세미오버핏 오버롤
여름 한정 가죽 소재 화이트 오버롤
여름에 어울리는 화이트 오버롤
오버핏 실루엣의 스웨이드 오버롤
트렌디한 모던룩, 레귤러핏 카디건 in 간절기
트렌디한 스트리트룩, 루즈핏 오버롤 in 봄
트렌디한 캐주얼룩, 레귤러핏 셔츠 in 봄
필수템! 스트리트 무드의 간절기용 오버롤
필수템! 캐주얼 무드의 겨울용 셔츠

User ID: 9718
-----------------------------------
가을 한정 울 소재 화이트 오버롤
가을에 어울리는 화이트 오버롤
간

## Item Similarity

In [11]:
item_metadata_path = Path(base_dir).joinpath(
    f"data/dataset/{DOMAIN}/item_metadata.parquet"
)
df_item = pd.read_parquet(item_metadata_path)

In [12]:
# item_vectors: (num_items, dim) numpy float32 배열이라고 가정
item_index = build_index(item_vectors)
item_idx = 0
query_vector = item_vectors[item_idx]
result = search(query_vector, item_index, top_k=5)
print(result)  # 예: {1: 0.9, 2: 0.8, ...}

{0: 0.9999997019767761, 4257: 0.9904316663742065, 5660: 0.9901280403137207, 4546: 0.49331796169281006, 5731: 0.45588570833206177}


In [13]:
item_id = item_id_maps[item_idx]

print(f"Item ID: {item_id}")
print(df_item[df_item["item_id"] == item_id]["title"].item())

Item ID: 1
겨울 감성 세미오버핏 나일론 셔츠


In [14]:
similar_indies = list(result.keys())[1:]
similar_item_ids = [item_id_maps[i] for i in similar_indies]

for item_id in similar_item_ids:
    print(f"\nItem ID: {item_id}")
    print("-----------------------------------")
    for t in sorted(df_item[df_item["item_id"] == item_id]["title"]):
        print(t)


Item ID: 4259
-----------------------------------
네이비 컬러의 모던 스타일 울 셔츠

Item ID: 5663
-----------------------------------
세미오버핏 실루엣의 플리스 셔츠

Item ID: 4549
-----------------------------------
필수템! 스포티 무드의 봄용 조끼

Item ID: 5734
-----------------------------------
슬림핏 실루엣의 나일론 슬랙스


# Recommendation
- User to Item 상품 추천

In [15]:
user_idx = 0
query_vector = user_vectors[user_idx]
result = search(query_vector, item_index, top_k=10)

item_indies = list(result.keys())[1:]

recommendation = {}
for i in item_indies:
    item_id = item_id_maps[i]
    recommendation[item_id] = result[i]

recommendation

{6405: 0.6897738575935364,
 7072: 0.6837248802185059,
 9525: 0.6798332333564758,
 6350: 0.6674239635467529,
 441: 0.6613531112670898,
 790: 0.6573948860168457,
 5568: 0.6317204236984253,
 7678: 0.61257004737854,
 3123: 0.590832531452179}

In [16]:
df_recommendation = df_item[
    df_item["item_id"].isin(list(recommendation.keys()))
].reset_index(drop=True)
df_recommendation["score"] = df_recommendation["item_id"].map(recommendation)
df_recommendation = df_recommendation.sort_values(
    "score", ascending=False, ignore_index=True
)

In [17]:
df[df["user_id"] == user_id_maps[user_idx]].reset_index(drop=True)

,user_id,item_id,timestamp,action,age,gender,title,color,style,fit,material,season,sleeve,category
0,1,3507,2025-09-21 03:01:00,click,46,F,슬림핏 실루엣의 데님 오버롤,레드,스포티,슬림핏,데님,겨울,롱슬리브,오버롤
1,1,9867,2025-09-21 03:02:00,click,46,F,봄 슬림핏 레드 나일론 롱슬리브 오버롤,레드,스포티,슬림핏,나일론,봄,롱슬리브,오버롤
2,1,3471,2025-09-02 00:05:00,click,46,F,"간절기 시즌, 레트로 무드의 오렌지 퍼 롱슬리브 루즈핏 원피스",오렌지,레트로,루즈핏,퍼,간절기,롱슬리브,원피스
3,1,6462,2025-09-02 00:09:00,click,46,F,"트렌디한 레트로룩, 슬림핏 원피스 in 여름",오렌지,레트로,슬림핏,스웨이드,여름,숏슬리브,원피스
4,1,441,2025-09-19 08:51:00,click,46,F,"봄 시즌, 캐주얼 무드의 와인 스웨이드 롱슬리브 루즈핏 슬랙스",와인,캐주얼,루즈핏,스웨이드,봄,롱슬리브,슬랙스
5,1,790,2025-09-19 08:54:00,click,46,F,필수템! 캐주얼 무드의 겨울용 슬랙스,와인,캐주얼,슬림핏,데님,겨울,롱,슬랙스
6,1,5568,2025-09-19 08:59:00,click,46,F,필수템! 캐주얼 무드의 봄용 슬랙스,와인,캐주얼,슬림핏,스웨이드,봄,민소매,슬랙스
7,1,731,2025-09-28 11:38:00,click,46,F,여름 한정 가죽 소재 화이트 오버롤,화이트,스트리트,루즈핏,가죽,여름,롱,오버롤
8,1,6652,2025-09-28 11:41:00,click,46,F,"트렌디한 스트리트룩, 루즈핏 오버롤 in 봄",화이트,스트리트,루즈핏,울,봄,7부,오버롤
9,1,7678,2025-09-28 11:40:00,click,46,F,여름에 어울리는 화이트 오버롤,화이트,스트리트,오버핏,레이온,여름,7부,오버롤


In [18]:
df_recommendation

,item_id,title,color,style,fit,material,season,sleeve,category,score
0,6405,겨울 감성 루즈핏 레이온 청바지,와인,러블리,루즈핏,레이온,겨울,숏,청바지,0.689774
1,7072,겨울 감성 루즈핏 가죽 청바지,와인,러블리,루즈핏,가죽,겨울,민소매,청바지,0.683725
2,9525,봄 한정 스웨이드 소재 와인 청바지,와인,러블리,루즈핏,스웨이드,봄,롱,청바지,0.679833
3,6350,간절기 감성 크롭핏 퍼 청바지,와인,러블리,크롭핏,퍼,간절기,롱슬리브,청바지,0.667424
4,441,"봄 시즌, 캐주얼 무드의 와인 스웨이드 롱슬리브 루즈핏 슬랙스",와인,캐주얼,루즈핏,스웨이드,봄,롱슬리브,슬랙스,0.661353
5,790,필수템! 캐주얼 무드의 겨울용 슬랙스,와인,캐주얼,슬림핏,데님,겨울,롱,슬랙스,0.657395
6,5568,필수템! 캐주얼 무드의 봄용 슬랙스,와인,캐주얼,슬림핏,스웨이드,봄,민소매,슬랙스,0.631720
7,7678,여름에 어울리는 화이트 오버롤,화이트,스트리트,오버핏,레이온,여름,7부,오버롤,0.612570
8,3123,필수템! 스트리트 무드의 겨울용 오버롤,화이트,스트리트,레귤러핏,나일론,겨울,민소매,오버롤,0.590833
